# Final Submission Notebook

This notebook generates the final submission file using the best models and pipeline developed.

## Pipeline:
1.  **Classifier:** 5-fold ensemble of `EfficientNet-B5` models trained on 3-channel PNGs.
2.  **Detector:** 5-fold ensemble of `YOLOv5s` models trained on 1-channel PNGs.
3.  **Post-Processing:** Classifier-guided filtering. Bounding boxes are removed from studies where the classifier predicts `Negative for Pneumonia` with a confidence greater than a set threshold (0.6).
4.  **Ensembling:**
    *   Classifier: Average the softmax outputs of the 5 models.
    *   Detector: For each image, concatenate predictions from all 5 models and then apply Non-Maximum Suppression (NMS) to get the final set of boxes.

## Steps:
1.  **Setup:** Load libraries and define configurations.
2.  **Classifier Inference:** Generate study-level predictions for the test set.
3.  **Detector Inference:** Generate image-level bounding box predictions for the test set.
4.  **Combine & Post-Process:** Merge classifier and detector results, apply filtering.
5.  **Format & Save:** Create `submission.csv` in the required format.

In [1]:
# --- 1. Setup: Imports and Configuration ---

import os
import sys
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import timm
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import glob
from torchvision.ops import nms

# --- Configuration ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Classifier Config
CLF_MODEL_NAME = 'tf_efficientnet_b5_ns'
CLF_IMG_SIZE = 512
CLF_BATCH_SIZE = 16
CLF_MODEL_PATHS = sorted(glob.glob('classifier_fold*_best.pth'))
CLASSES = ['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
CLASS_MAP_LOWER = {
    'Negative for Pneumonia': 'negative',
    'Typical Appearance': 'typical',
    'Indeterminate Appearance': 'indeterminate',
    'Atypical Appearance': 'atypical'
}

# Detector Config
DET_IMG_SIZE = 640
DET_BATCH_SIZE = 16
DET_MODEL_PATHS = sorted(glob.glob('yolov5_runs/train_cv/yolov5s_fold*/weights/best.pt'))

# Post-Processing & Ensemble Config
POST_PROCESS_CONF_THRESHOLD = 0.6 # For filtering boxes based on 'Negative' class pred
DETECTOR_CONF_THRESHOLD = 0.001 # Initial confidence threshold for detector predictions
NMS_IOU_THRESHOLD = 0.5 # IOU threshold for NMS

# Data Paths
TEST_IMAGE_DIR_3CH = 'test_png_3ch/' # For classifier
TEST_IMAGE_DIR_1CH = 'test_png/'     # For detector
SAMPLE_SUB_PATH = 'sample_submission.csv'

print(f"Using device: {DEVICE}")
print(f"Found {len(CLF_MODEL_PATHS)} classifier models.")
print(f"Found {len(DET_MODEL_PATHS)} detector models.")

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Found 5 classifier models.
Found 5 detector models.


In [3]:
# --- 2. Classifier Inference (5-Fold Ensemble) ---

def get_transforms(img_size):
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

class SIIMCOVIDTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image, os.path.basename(image_path).replace('.png', '')

print("--- Starting Classifier Inference ---")

# Load test file list
df_test_imgs = pd.DataFrame({'id': os.listdir(TEST_IMAGE_DIR_3CH)})
df_test_imgs['image_id'] = df_test_imgs['id'].str.replace('.png', '')
df_test_imgs['path'] = TEST_IMAGE_DIR_3CH + df_test_imgs['id']

# Create dataset and dataloader
test_dataset = SIIMCOVIDTestDataset(df_test_imgs['path'].values, transform=get_transforms(CLF_IMG_SIZE))
test_loader = DataLoader(test_dataset, batch_size=CLF_BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# Load all 5 classifier models
models = []
for path in CLF_MODEL_PATHS:
    model = timm.create_model(CLF_MODEL_NAME, pretrained=False, num_classes=len(CLASSES))
    model.load_state_dict(torch.load(path))
    model.to(DEVICE)
    model.eval()
    models.append(model)
print(f"Loaded {len(models)} classifier models onto {DEVICE}")

# Run inference
all_preds = []
with torch.no_grad():
    for images, image_ids in tqdm(test_loader, desc="Classifier Inference"):
        images = images.to(DEVICE)
        
        # Get predictions from all models and average them
        batch_preds = torch.zeros((images.size(0), len(CLASSES)), device=DEVICE)
        for model in models:
            batch_preds += torch.softmax(model(images), dim=1)
        batch_preds /= len(models)
        
        for i, image_id in enumerate(image_ids):
            preds = {f'pred_{cls}': batch_preds[i, j].item() for j, cls in enumerate(CLASSES)}
            preds['image_id'] = image_id
            all_preds.append(preds)

df_image_preds = pd.DataFrame(all_preds)

# Aggregate to study level
# FIX: Create the image_id to StudyInstanceUID mapping by walking the test directory
test_dcm_paths = glob.glob('test/*/*/*.dcm')
test_map_data = []
for path in test_dcm_paths:
    parts = path.split('/')
    study_id = parts[1]
    image_id = parts[-1].replace('.dcm', '')
    test_map_data.append({'image_id': image_id, 'StudyInstanceUID': study_id})
study_id_map = pd.DataFrame(test_map_data).drop_duplicates()

df_image_preds_with_study = df_image_preds.merge(study_id_map, on='image_id', how='left')

pred_cols = [f'pred_{c}' for c in CLASSES]
df_study_preds = df_image_preds_with_study.groupby('StudyInstanceUID')[pred_cols].mean().reset_index()

print("Classifier inference complete. Study-level predictions created.")
display(df_study_preds.head())

--- Starting Classifier Inference ---


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


Loaded 5 classifier models onto cuda


Classifier Inference:   0%|          | 0/40 [00:00<?, ?it/s]

Classifier Inference:   2%|▎         | 1/40 [00:02<01:32,  2.37s/it]

...

Classifier Inference:  98%|█████████▊| 39/40 [00:30<00:00,  1.36it/s]

Classifier Inference: 100%|██████████| 40/40 [00:30<00:00,  1.41it/s]

Classifier Inference: 100%|██████████| 40/40 [00:30<00:00,  1.29it/s]

Classifier inference complete. Study-level predictions created.


,StudyInstanceUID,pred_Negative for Pneumonia,pred_Typical Appearance,pred_Indeterminate Appearance,pred_Atypical Appearance
0,000c9c05fd14,0.248463,0.530363,0.144147,0.077027
1,00c74279c5b7,0.322444,0.404008,0.185479,0.088069
2,00ccd633fb0e,0.481045,0.307503,0.120407,0.091045
3,00e936c58da6,0.140884,0.732224,0.086526,0.040365
4,01206a422293,0.055123,0.804715,0.094880,0.045282


In [4]:
# --- 3. Detector Inference (5-Fold Ensemble) ---

print("--- Starting Detector Inference ---")

# Load all 5 detector models
det_models = []
for path in tqdm(DET_MODEL_PATHS, desc="Loading detector models"):
    # Use torch.hub.load to get the YOLOv5 model
    # Set force_reload=True to ensure the correct weights are loaded for each fold
    model = torch.hub.load(
        'ultralytics/yolov5',
        'custom',
        path=path,
        force_reload=True,
        _verbose=False # Suppress verbose output for cleaner logs
    )
    model.to(DEVICE).eval()
    model.conf = DETECTOR_CONF_THRESHOLD
    model.iou = NMS_IOU_THRESHOLD # This is for the model's internal NMS, we will do a final NMS later
    det_models.append(model)
print(f"Loaded {len(det_models)} detector models onto {DEVICE}")

# Get test image paths
test_image_paths = sorted(glob.glob(f'{TEST_IMAGE_DIR_1CH}/*.png'))
test_image_ids = [os.path.basename(p).replace('.png', '') for p in test_image_paths]
print(f"Found {len(test_image_paths)} test images for detection.")

# Run inference and collect all raw predictions from all models
all_det_preds = []
with torch.no_grad():
    for i in tqdm(range(0, len(test_image_paths), DET_BATCH_SIZE), desc="Detector Inference"):
        batch_paths = test_image_paths[i:i+DET_BATCH_SIZE]
        batch_image_ids = test_image_ids[i:i+DET_BATCH_SIZE]

        # Store all predictions for this batch from all models
        batch_model_preds = [[] for _ in range(len(batch_paths))]

        for model in det_models:
            results = model(batch_paths, size=DET_IMG_SIZE)
            preds_df_list = results.pandas().xyxy
            
            for j, preds_df in enumerate(preds_df_list):
                if not preds_df.empty:
                    batch_model_preds[j].append(preds_df[['xmin', 'ymin', 'xmax', 'ymax', 'confidence']].values)

        # For each image in the batch, combine the predictions into the final list
        for j, image_id in enumerate(batch_image_ids):
            if batch_model_preds[j]:
                combined_preds = np.vstack(batch_model_preds[j])
                for pred in combined_preds:
                    all_det_preds.append({
                        'image_id': image_id,
                        'x_min': pred[0],
                        'y_min': pred[1],
                        'x_max': pred[2],
                        'y_max': pred[3],
                        'confidence': pred[4]
                    })

df_det_preds_raw = pd.DataFrame(all_det_preds)
print("Detector inference complete. Raw box predictions created.")
display(df_det_preds_raw.head())

--- Starting Detector Inference ---


Loading detector models:   0%|          | 0/5 [00:00<?, ?it/s]

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /app/.cache/torch/hub/master.zip


Loading detector models:  20%|██        | 1/5 [00:01<00:05,  1.49s/it]

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /app/.cache/torch/hub/master.zip


...

/app/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


/app/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
Detector Inference: 100%|██████████| 40/40 [02:19<00:00,  3.07s/it]

Detector Inference: 100%|██████████| 40/40 [02:19<00:00,  3.49s/it]

Detector inference complete. Raw box predictions created.


,image_id,x_min,y_min,x_max,y_max,confidence
0,004cbd797cd1,2571.596191,957.954956,2992.000000,1895.753540,0.032310
1,004cbd797cd1,0.000000,1076.239868,449.762543,2061.799805,0.027573
2,004cbd797cd1,2289.915283,1102.464844,2983.130127,2006.620483,0.027098
3,004cbd797cd1,3.934152,1187.782593,715.708008,2221.086182,0.022080
4,004cbd797cd1,1757.265015,1090.640137,2578.268555,2020.082031,0.018571


In [5]:
# --- 4. Combine, Post-Process, and Format for Submission ---

print("--- Combining, Post-Processing, and Formatting ---")

# --- Step 4.1: Apply NMS and Classifier-Guided Filtering ---
final_image_preds = []

# Merge raw detector predictions with study info to get StudyInstanceUID
df_det_preds_raw_with_study = df_det_preds_raw.merge(study_id_map, on='image_id', how='left')

# Merge with study-level classifier predictions
df_det_merged = df_det_preds_raw_with_study.merge(df_study_preds, on='StudyInstanceUID', how='left')

# Apply classifier-guided filtering
print(f"Boxes before classifier filtering: {len(df_det_merged)}")
df_det_filtered = df_det_merged[df_det_merged['pred_Negative for Pneumonia'] < POST_PROCESS_CONF_THRESHOLD]
print(f"Boxes after classifier filtering: {len(df_det_filtered)}")


# Group by image to apply NMS
for image_id, group in tqdm(df_det_filtered.groupby('image_id'), desc="Applying NMS"):
    boxes = torch.tensor(group[['x_min', 'y_min', 'x_max', 'y_max']].values, dtype=torch.float32)
    scores = torch.tensor(group['confidence'].values, dtype=torch.float32)
    
    # Apply NMS
    keep_indices = nms(boxes, scores, NMS_IOU_THRESHOLD)
    
    final_boxes = boxes[keep_indices].cpu().numpy()
    final_scores = scores[keep_indices].cpu().numpy()
    
    # Format for submission string
    pred_string_parts = []
    for box, score in zip(final_boxes, final_scores):
        pred_string_parts.append(f"opacity {score:.4f} {box[0]:.1f} {box[1]:.1f} {box[2]:.1f} {box[3]:.1f}")
    
    if pred_string_parts:
        final_image_preds.append({
            'id': f"{image_id}_image",
            'PredictionString': " ".join(pred_string_parts)
        })

df_image_sub = pd.DataFrame(final_image_preds)

# --- Step 4.2: Format Study-Level Predictions ---
study_sub_list = []
for _, row in df_study_preds.iterrows():
    study_id = row['StudyInstanceUID']
    pred_strings = []
    for cls in CLASSES:
        pred_strings.append(f"{CLASS_MAP_LOWER[cls]} {row[f'pred_{cls}']:.4f} 0 0 1 1")
    
    study_sub_list.append({
        'id': f"{study_id}_study",
        'PredictionString': " ".join(pred_strings)
    })

df_study_sub = pd.DataFrame(study_sub_list)

# --- Step 4.3: Combine and Create Final Submission File ---
df_submission = pd.concat([df_study_sub, df_image_sub], ignore_index=True)

# Add entries for images with no predicted boxes (submit 'none 1 0 0 1 1')
all_test_image_ids = {f"{img_id}_image" for img_id in test_image_ids}
predicted_image_ids = set(df_image_sub['id'].unique())
missing_image_ids = all_test_image_ids - predicted_image_ids

missing_df = pd.DataFrame([{'id': img_id, 'PredictionString': 'none 1 0 0 1 1'} for img_id in missing_image_ids])
df_submission = pd.concat([df_submission, missing_df], ignore_index=True)

# Reorder to match sample submission
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
df_submission = df_submission.set_index('id').reindex(sample_sub['id']).reset_index()

# Save to file
df_submission.to_csv('submission.csv', index=False)

print("\nFinal submission.csv created successfully!")
display(df_submission.head())
display(df_submission.tail())
print(f"Total rows in submission: {len(df_submission)}")

--- Combining, Post-Processing, and Formatting ---
Boxes before classifier filtering: 406458
Boxes after classifier filtering: 388882


Applying NMS:   0%|          | 0/611 [00:00<?, ?it/s]

Applying NMS:  14%|█▍        | 86/611 [00:00<00:00, 852.39it/s]

Applying NMS:  29%|██▉       | 180/611 [00:00<00:00, 898.83it/s]

Applying NMS:  44%|████▍     | 270/611 [00:00<00:00, 894.59it/s]

...

Applying NMS: 100%|██████████| 611/611 [00:00<00:00, 904.65it/s]


Final submission.csv created successfully!


,id,PredictionString
0,000c9c05fd14_study,negative 0.2485 0 0 1 1 typical 0.5304 0 0 1 1...
1,00c74279c5b7_study,negative 0.3224 0 0 1 1 typical 0.4040 0 0 1 1...
2,00ccd633fb0e_study,negative 0.4810 0 0 1 1 typical 0.3075 0 0 1 1...
3,00e936c58da6_study,negative 0.1409 0 0 1 1 typical 0.7322 0 0 1 1...
4,01206a422293_study,negative 0.0551 0 0 1 1 typical 0.8047 0 0 1 1...


,id,PredictionString
1239,ff03d1d41968_image,opacity 0.0481 0.0 718.3 669.3 2132.0 opacity ...
1240,ff0743bee789_image,opacity 0.0560 0.0 432.0 392.3 1201.1 opacity ...
1241,ffab0f8f27f0_image,opacity 0.0519 0.0 547.8 494.1 1516.6 opacity ...
1242,ffbeafe30b77_image,opacity 0.0678 0.9 414.9 373.0 1134.9 opacity ...
1243,ffe942c8655f_image,opacity 0.0473 1179.5 256.2 1387.0 671.9 opaci...


Total rows in submission: 1244
